In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_predict, cross_validate, GridSearchCV

from sklearn.preprocessing import RobustScaler, PowerTransformer

import itertools

from sklearn.metrics.pairwise import paired_distances

import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from sklearn.compose import ColumnTransformer
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

from neo4j import GraphDatabase

In [2]:
df = pd.read_excel('/home/guest/Загрузки/Basheer_demo.xlsx')

In [5]:
vocs_names = [x.replace('l0_', '') for x in df.columns if 'l0_' in x][:5]

df_vocs = df[[x for x in df.columns if 'l0_' in x][:5]]
df_vocs.columns = vocs_names

unique_probe_id = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['PatientID'] = df['Cardio_Qvark_ID']
df_vocs['ProbeId'] = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['DateTime'] = [datetime.now() for x in range(df.shape[0])]

y = (df['Myocardial_perfusion_defect_after_stress_ATP'] == 'Yes').astype(float)
df_vocs['target'] =y.replace([0, 1], ['Healthy', 'ИБС'])

X_train, _, y_train, _ = train_test_split(df_vocs.drop('target', axis = 'columns'), y, test_size=0.8, random_state=42)
X_train['target_short_name'] =y_train.replace([0, 1], ['Healthy', 'ИБС'])
X_train['target_icd_10'] =y_train.replace([0, 1], ['Healthy', 'I20.8'])
X_train['target_icd_11'] =y_train.replace([0, 1], ['Healthy', 'BA40.1'])

df_vocs = X_train

df1 = df_vocs

print(df_vocs.to_markdown())


|    |   42_03257085331836 |   43_04741898699762 |   44_01450529149254 |   44_0489926172728 |   44_99144370755021 |   PatientID | ProbeId    | DateTime                   | target_short_name   | target_icd_10   | target_icd_11   |
|---:|--------------------:|--------------------:|--------------------:|-------------------:|--------------------:|------------:|:-----------|:---------------------------|:--------------------|:----------------|:----------------|
| 59 |           0.0815782 |            0.894392 |           0.0659017 |          0.0497411 |             3.30478 |       19947 | ProbeId_59 | 2024-08-31 11:14:48.035966 | Healthy             | Healthy         | Healthy         |
| 63 |           0.0555983 |            0.727569 |           0.0786207 |          0.0534634 |             4.49773 |       21255 | ProbeId_63 | 2024-08-31 11:14:48.035968 | Healthy             | Healthy         | Healthy         |
| 69 |           1.74384   |            3.37286  |           0.0830686 |        

In [142]:
vocs_names = [x.replace('l0_', '') for x in df.columns if 'l0_' in x][4:10]

df_vocs = df[[x for x in df.columns if 'l0_' in x][4:10]]
df_vocs.columns = vocs_names

unique_probe_id = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['PatientID'] = df['Cardio_Qvark_ID']
df_vocs['ProbeId'] = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['DateTime'] = [datetime.now() for x in range(df.shape[0])]

y = (df['Myocardial_perfusion_defect_after_stress_ATP'] == 'Yes').astype(float)
df_vocs['target'] =y.replace([0, 1], ['Healthy', 'ИБС'])

X_train, _, y_train, _ = train_test_split(df_vocs.drop('target', axis = 'columns'), y, test_size=.9, random_state=58)
X_train['target_short_name'] =y_train.replace([0, 1], ['Healthy', 'ИБС'])
X_train['target_icd_10'] =y_train.replace([0, 1], ['Healthy', 'I20.8'])
X_train['target_icd_11'] =y_train.replace([0, 1], ['Healthy', 'BA40.1'])

df_vocs = X_train

df2 = df_vocs

print(df_vocs.to_markdown())

|    |   44_99144370755021 |   45_99197266970869 |   47_04077398412186 |   48_045099177099885 |   49_00477743760921 |   49_99501251131184 |   PatientID | ProbeId    | DateTime                   | target_short_name   | target_icd_10   | target_icd_11   |
|---:|--------------------:|--------------------:|--------------------:|---------------------:|--------------------:|--------------------:|------------:|:-----------|:---------------------------|:--------------------|:----------------|:----------------|
| 76 |             2.97995 |           0.0584557 |            8.01259  |           0.178858   |           0.0308905 |          0.00896429 |       22015 | ProbeId_76 | 2024-08-30 19:37:52.812337 | Healthy             | Healthy         | Healthy         |
| 79 |             1.92739 |           0.0499004 |            0.192831 |           0.00517879 |           0.0379916 |          0.00480427 |       23843 | ProbeId_79 | 2024-08-30 19:37:52.812339 | Healthy             | Healthy         | He

In [8]:
import json
from datetime import datetime
from typing import List, Optional, Union
from dataclasses import dataclass, field
import pandas as pd
from neo4j import GraphDatabase

@dataclass
class VOC:
    identifier: str = 'unknown'
    value: Optional[float] = None
    method: str = 'Silantiev_maximization'
    chemical_formula: Optional['Substance'] = None

@dataclass
class Target:
    short_name: str
    icd_10: str
    icd_11: str

@dataclass
class Person:
    person_id: str

@dataclass
class Volatome:
    unique_id: str
    person: Person
    collection_time: datetime = field(default_factory=datetime.now)
    breath_type: str = 'normal'
    normalization: Optional[str] = 'raw'
    targets: List[Target] = field(default_factory=list)
    voc_objects: List[VOC] = field(default_factory=list)

    def to_dict(self):
        return {
            'unique_id': self.unique_id,
            'person': self.person.__dict__,
            'collection_time': self.collection_time.isoformat(),
            'breath_type': self.breath_type,
            'normalization': self.normalization,
            'targets': [target.__dict__ for target in self.targets],
            'voc_objects': [voc.__dict__ for voc in self.voc_objects]
        }

@dataclass
class VolatomeVoid:
    volatome_storage: dict = field(default_factory=dict)

    def add_volatome(self, volatome: Volatome):
        self.volatome_storage[volatome.unique_id] = volatome

    def merge(self, other: 'VolatomeVoid'):
        for volatome in other.volatome_storage.values():
            if volatome.unique_id not in self.volatome_storage:
                self.add_volatome(volatome)

    def save_to_json(self, filename: str):
        with open(filename, 'w') as f:
            json.dump({uid: volatome.to_dict() for uid, volatome in self.volatome_storage.items()}, f)

class Neo4jDatabase:
    def __init__(self, uri: str, user: str, password: str):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_volatome(self, volatome: Volatome):
        with self.driver.session() as session:
            session.write_transaction(self._create_volatome, volatome)

    @staticmethod
    def _create_volatome(tx, volatome: Volatome):
        # Create Person node
        tx.run("MERGE (p:Person {person_id: $person_id})", person_id=volatome.person.person_id)

        # Create Volatome node
        tx.run("""
            CREATE (v:Volatome {
                unique_id: $unique_id,
                collection_time: $collection_time,
                breath_type: $breath_type,
                normalization: $normalization
            })
            """,
            unique_id=volatome.unique_id,
            collection_time=volatome.collection_time.isoformat(),
            breath_type=volatome.breath_type,
            normalization=volatome.normalization
        )

        # Create relationships
        tx.run("""
            MATCH (p:Person {person_id: $person_id}), (v:Volatome {unique_id: $unique_id})
            CREATE (p)-[:BREATHES]->(v)
            """,
            person_id=volatome.person.person_id,
            unique_id=volatome.unique_id
        )

        # Create VOC nodes and relationships
        for voc in volatome.voc_objects:
            tx.run("""
                CREATE (voc:VOC {
                    identifier: $identifier,
                    value: $value,
                    method: $method
                })
                """,
                identifier=voc.identifier,
                value=voc.value,
                method=voc.method
            )

            # Create relationship between Volatome and VOC
            tx.run("""
                MATCH (v:Volatome {unique_id: $unique_id}), (voc:VOC {identifier: $identifier})
                CREATE (v)-[:CONTAINS]->(voc)
                """,
                unique_id=volatome.unique_id,
                identifier=voc.identifier
            )

        # Create Target nodes and relationships
        for target in volatome.targets:
            tx.run("""
                CREATE (t:Target {
                    short_name: $short_name,
                    icd_10: $icd_10,
                    icd_11: $icd_11
                })
                """,
                short_name=target.short_name,
                icd_10=target.icd_10,
                icd_11=target.icd_11
            )

            # Create relationship between Volatome and Target
            tx.run("""
                MATCH (v:Volatome {unique_id: $unique_id}), (t:Target {short_name: $short_name})
                CREATE (v)-[:ASSOCIATED_WITH]->(t)
                """,
                unique_id=volatome.unique_id,
                short_name=target.short_name
            )

# Load Data
def load_data_from_tables():
    # Sample data for the first table
    data1 = {
        '526_03257085331836': [0.0347108, 0.936501, 0.0703742, 0.0917151, 0.741693, 0.074873, 0.106864, 0.171386],
        '742_04741898699762': [0.289125, 1.91298, 2.49884, 1.27432, 0.964586, 0.811691, 3.14315, 3.38432],
        '44_01450529149254': [0.0396336, 0.0739828, 0.0789201, 0.0940473, 0.0550934, 0.0709599, 0.0991804, 0.0795997],
        '49_0489926172728': [0.0231194, 0.087051, 0.101689, 0.0762481, 0.0501778, 0.0548712, 0.139365, 0.137908],
        '96_99144370755021': [3.52877, 3.51884, 2.68527, 5.33148, 3.35456, 3.9698, 4.15313, 2.83617],
        'PatientID': [17558, 21569, 21094, 19907, 20195, 21387, 20007, 18380],
        'ProbeId': ['ProbeId_2', 'ProbeId_23', 'ProbeId_74', 'ProbeId_20', 'ProbeId_60', 'ProbeId_71', 'ProbeId_14', 'ProbeId_51'],
        'DateTime': [
            '2024-08-30 23:45:47.877923', '2024-08-30 23:45:47.877936', 
            '2024-08-30 23:45:47.877968', '2024-08-30 23:45:47.877934', 
            '2024-08-30 23:45:47.877960', '2024-08-30 23:45:47.877967', 
            '2024-08-30 23:45:47.877930', '2024-08-30 23:45:47.877954'
        ],
        'target_short_name': ['ИБС', 'ИБС', 'Healthy', 'ИБС', 'Healthy', 'Healthy', 'ИБС', 'Healthy'],
        'target_icd_10': ['I20.8', 'I20.8', 'Healthy', 'I20.8', 'Healthy', 'Healthy', 'I20.8', 'Healthy'],
        'target_icd_11': ['BA40.1', 'BA40.1', 'Healthy', 'BA40.1', 'Healthy', 'Healthy', 'BA40.1', 'Healthy']
    }

    # Sample data for the second table
    data2 = {
        '44_99144370755021': [2.97995, 1.92739, 2.68572, 4.15313, 4.53717, 3.51884, 2.65582, 3.97766],
        '45_99197266970869': [0.0584557, 0.0499004, 0.0527857, 0.0827267, 0.0946463, 0.0708363, 0.046235, 0.0668665],
        '47_04077398412186': [8.01259, 0.192831, 3.64771, 6.01692, 0.298547, 0.166848, 0.119985, 0.130496],
        '48_045099177099885': [0.178858, 0.00517879, 0.0876438, 0.142089, 0.0110313, 0.00590104, 0.00359185, 0.00457821],
        '49_00477743760921': [0.0308905, 0.0379916, 0.0344578, 0.0984609, 0.0895525, 0.0090919, 0.0695031, 0.0493458],
        '49_99501251131184': [0.00896429, 0.00480427, 0.0064166, 0.0137742, 0.011893, 0.00815203, 0.00597724, 0.00639664],
        'PatientID': [22015, 23843, 21869, 20007, 21925, 21569, 17508, 17501],
        'ProbeId': ['ProbeId_76', 'ProbeId_79', 'ProbeId_21', 'ProbeId_14', 'ProbeId_25', 'ProbeId_23', 'ProbeId_32', 'ProbeId_35'],
        'DateTime': [
            '2024-08-30 19:25:48.903608', '2024-08-30 19:25:48.903611', 
            '2024-08-30 19:25:48.903569', '2024-08-30 19:25:48.903565', 
            '2024-08-30 19:25:48.903572', '2024-08-30 19:25:48.903571', 
            '2024-08-30 19:25:48.903577', '2024-08-30 19:25:48.903579'
        ],
        'target_short_name': ['Healthy', 'Healthy', 'ИБС', 'ИБС', 'ИБС', 'ИБС', 'Healthy', 'Healthy'],
        'target_icd_10': ['Healthy', 'Healthy', 'I20.8', 'I20.8', 'I20.8', 'I20.8', 'Healthy', 'Healthy'],
        'target_icd_11': ['Healthy', 'Healthy', 'BA40.1', 'BA40.1', 'BA40.1', 'BA40.1', 'Healthy', 'Healthy']
    }

    # Convert data to DataFrames
    df1 = pd.DataFrame(data1)
    df1['DateTime'] = pd.to_datetime(df1['DateTime'])  # Convert DateTime to datetime objects

    df2 = pd.DataFrame(data2)
    df2['DateTime'] = pd.to_datetime(df2['DateTime'])  # Convert DateTime to datetime objects

    # Create VolatomeVoid objects from both DataFrames
    volatome_void_1 = VolatomeVoid()
    volatome_void_2 = VolatomeVoid()

    # Load data into VolatomeVoid 1
    for index, row in df1.iterrows():
        person = Person(person_id=str(row['PatientID']))
        voc_objects = [
            VOC(identifier=col, value=row[col]) 
            for col in df1.columns[:-5]  # Exclude PatientID, ProbeId, DateTime, target_short_name, target_icd_10, target_icd_11
        ]
        target = Target(short_name=row['target_short_name'], icd_10=row['target_icd_10'], icd_11=row['target_icd_11'])
        volatome = Volatome(
            unique_id=row['ProbeId'],
            person=person,
            collection_time=row['DateTime'],
            targets=[target],
            voc_objects=voc_objects
        )
        volatome_void_1.add_volatome(volatome)

    # Load data into VolatomeVoid 2
    for index, row in df2.iterrows():
        person = Person(person_id=str(row['PatientID']))
        voc_objects = [
            VOC(identifier=col, value=row[col]) 
            for col in df2.columns[:-5]  # Exclude PatientID, ProbeId, DateTime, target_short_name, target_icd_10, target_icd_11
        ]
        target = Target(short_name=row['target_short_name'], icd_10=row['target_icd_10'], icd_11=row['target_icd_11'])
        volatome = Volatome(
            unique_id=row['ProbeId'],
            person=person,
            collection_time=row['DateTime'],
            targets=[target],
            voc_objects=voc_objects
        )
        volatome_void_2.add_volatome

In [19]:
volatome_void_2.volatome_storage

{'ProbeId_76': Volatome(unique_id='ProbeId_76', person=Person(person_id='22015'), collection_time=Timestamp('2024-08-30 19:25:48.903608'), breath_type='normal', normalization='raw', targets=[Target(short_name='Healthy', icd_10='Healthy', icd_11='Healthy')], voc_objects=[VOC(identifier='44_99144370755021', value=2.97995, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='45_99197266970869', value=0.0584557, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='47_04077398412186', value=8.01259, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='48_045099177099885', value=0.178858, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='49_00477743760921', value=0.0308905, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='49_99501251131184', value=0.00896429, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='PatientID', value=22015, method='Silantiev_maximization'

In [23]:
import pandas as pd
from neo4j import GraphDatabase

# Define the Neo4j connection parameters
NEO4J_URI = "neo4j://localhost:7687"  # Change if needed
NEO4J_USER = ""  # Your username
NEO4J_PASSWORD = ""  # Your password

# Load the data into a DataFrame
data = {
    "526_03257085331836": [0.0347108, 0.936501, 0.0703742, 0.0917151, 0.741693, 0.074873, 0.106864, 0.171386],
    "742_04741898699762": [0.289125, 1.91298, 2.49884, 1.27432, 0.964586, 0.811691, 3.14315, 3.38432],
    "44_01450529149254": [0.0396336, 0.0739828, 0.0789201, 0.0940473, 0.0550934, 0.0709599, 0.0991804, 0.0795997],
    "49_0489926172728": [0.0231194, 0.087051, 0.101689, 0.0762481, 0.0501778, 0.0548712, 0.139365, 0.137908],
    "96_99144370755021": [3.52877, 3.51884, 2.68527, 5.33148, 3.35456, 3.9698, 4.15313, 2.83617],
    "PersonID": [17558, 21569, 21094, 19907, 20195, 21387, 20007, 18380],
    "ProbeId": ["ProbeId_2", "ProbeId_23", "ProbeId_74", "ProbeId_20", "ProbeId_60", "ProbeId_71", "ProbeId_14", "ProbeId_51"],
    "DateTime": [
        "2024-08-30 23:45:47.877923", "2024-08-30 23:45:47.877936", "2024-08-30 23:45:47.877968",
        "2024-08-30 23:45:47.877934", "2024-08-30 23:45:47.877960", "2024-08-30 23:45:47.877967",
        "2024-08-30 23:45:47.877930", "2024-08-30 23:45:47.877954"
    ],
    "target_short_name": ["ИБС", "ИБС", "Healthy", "ИБС", "Healthy", "Healthy", "ИБС", "Healthy"],
    "target_icd_10": ["I20.8", "I20.8", "Healthy", "I20.8", "Healthy", "Healthy", "I20.8", "Healthy"],
    "target_icd_11": ["BA40.1", "BA40.1", "Healthy", "BA40.1", "Healthy", "Healthy", "BA40.1", "Healthy"]
}

df = pd.DataFrame(data)

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def create_graph(tx, person_id, probe_id, date_time, breathing_type, normed, vocs, target):
    # Create Person node
    tx.run("MERGE (p:Person {PersonID: $person_id})", person_id=person_id)

    # Create Volatome node
    tx.run("MERGE (v:Volatome {ProbeId: $probe_id}) "
           "SET v.DateTime = $date_time, v.BreathingType = $breathing_type, v.Normed = $normed",
           probe_id=probe_id, date_time=date_time, breathing_type='Normal', normed='Raw')

    # Create VOC nodes and establish relationships
    for voc_name, voc_value in vocs.items():
        tx.run("MERGE (vo:VOC {Name: $voc_name}) "
               "SET vo.Value = $voc_value", voc_name=voc_name, voc_value=voc_value)
        tx.run("MATCH (v:Volatome {ProbeId: $probe_id}), (vo:VOC {Name: $voc_name}) "
               "MERGE (v)-[:CONTAINS]->(vo)", probe_id=probe_id, voc_name=voc_name)

    # Create Target node
    tx.run("MERGE (t:Target {ShortName: $short_name, ICD10: $icd10, ICD11: $icd11})", 
           short_name=target['short_name'], icd10=target['icd_10'], icd11=target['icd_11'])
    tx.run("MATCH (v:Volatome {ProbeId: $probe_id}), (t:Target {ShortName: $short_name, ICD10: $icd10, ICD11: $icd11}) "
           "MERGE (v)-[:ASSOCIATED_WITH]->(t)", 
           probe_id=probe_id, short_name=target['short_name'], icd10=target['icd_10'], icd11=target['icd_11'])

    # Establish relationship between Person and Volatome
    tx.run("MATCH (p:Person {PersonID: $person_id}), (v:Volatome {ProbeId: $probe_id}) "
           "MERGE (p)-[:BREATHES]->(v)", person_id=person_id, probe_id=probe_id)

# Import data into Neo4j
with driver.session() as session:
    for index, row in df.iterrows():
        vocs = {
            '526_03257085331836': row['526_03257085331836'],
            '742_04741898699762': row['742_04741898699762'],
            '44_01450529149254': row['44_01450529149254'],
            '49_0489926172728': row['49_0489926172728'],
            '96_99144370755021': row['96_99144370755021']
        }
        target = {
            'short_name': row['target_short_name'],
            'icd_10': row['target_icd_10'],
            'icd_11': row['target_icd_11']
        }
        session.write_transaction(create_graph, row['PersonID'], row['ProbeId'], row['DateTime'], 'Normal', 'Raw', vocs, target)

# Close the driver
driver.close()